In [2]:
%pip install numbers-parser


  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from numbers_parser import Document
import pandas as pd
from pathlib import Path

numbers_path = Path("../map_data_demo.numbers").resolve()
output_path = numbers_path.with_suffix(".csv")

print(f"Reading Numbers file: {numbers_path}")
doc = Document(numbers_path)

sheet = doc.sheets[0]
table = sheet.tables[0]

rows = [row for row in table.rows(values_only=True) if any(cell is not None for cell in row)]
if not rows:
    raise ValueError("The first table in the Numbers file appears to be empty.")

header, *body = rows

df = pd.DataFrame(body, columns=header)
df.to_csv(output_path, index=False)

print(f"Exported CSV: {output_path}")
df.head()


Reading Numbers file: /Users/ujwalneethipudi/Downloads/European Political Tech Landscape/map_data_demo.numbers
Exported CSV: /Users/ujwalneethipudi/Downloads/European Political Tech Landscape/map_data_demo.csv


,S. No.,Entity,Logo,HQ,Domain,Description,Map Bucket 1,Map Bucket 2,Relevance Flag (Y/N),Sub Bucket,Tools/Products,Map Bucket 1 (Normalized)
0,1.0,Panterra Global,None,United Kingdom,https://www.panterra.global/,Strategic advisory to governments/corporations...,Media & Messaging,None,Y,None,None,Messaging & Media
1,3.0,influencethe.com,None,Bulgaria,https://influencethe.com/,Disinformation detection & “community intellig...,Media & Messaging,None,Y,Information Integrity / Disinformation,None,Messaging & Media
2,4.0,Electify,None,None,None,"Secure online election software; customizable,...",Voting Tech,None,Y,None,None,Voting Tech
3,5.0,Genaios GmbH,None,Germany,https://genaios.ai/,"AI-powered fact-checking, content originality ...",Media & Messaging,None,Y,None,None,Messaging & Media
4,14.0,Pakflatt,None,United Kingdom,https://www.pakflatt.com/,Manufacturer of polling booths and election-eq...,GovTech / Civic Infrastructure,Voting Tech,Y,None,None,GovTech / Civic Infrastructure


In [6]:
# Export grouped JSON for the app
import json
from pathlib import Path

# Columns expected in CSV
# 'Entity', 'Domain', 'Description', 'Map Bucket 1 (Normalized)'

csv_path = Path("../map_data_demo.csv").resolve()
src_json_out = Path("../src/data/companies.json").resolve()
public_json_out = Path("../public/companies.json").resolve()
src_json_out.parent.mkdir(parents=True, exist_ok=True)
public_json_out.parent.mkdir(parents=True, exist_ok=True)

print(f"Reading CSV: {csv_path}")
df = pd.read_csv(csv_path)

required_cols = [
    'Entity',
    'Domain',
    'Description',
    'Map Bucket 1 (Normalized)'
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

# Clean column names for consistency
clean = df[required_cols].copy()
clean = clean.rename(columns={
    'Entity': 'name',
    'Domain': 'domain',
    'Description': 'description',
    'Map Bucket 1 (Normalized)': 'category'
})

# Drop fully empty names or categories
clean = clean.dropna(subset=['name', 'category'])

# Group by category
grouped = {}
for cat, group in clean.groupby('category'):
    items = []
    for _, row in group.iterrows():
        item = {
            'name': str(row['name']).strip(),
            'domain': (str(row['domain']).strip() if pd.notna(row['domain']) else ''),
            'description': (str(row['description']).strip() if pd.notna(row['description']) else '')
        }
        items.append(item)
    grouped[cat] = items

for destination in [src_json_out, public_json_out]:
    with open(destination, 'w', encoding='utf-8') as f:
        json.dump(grouped, f, ensure_ascii=False, indent=2)
    print(f"Wrote JSON: {destination}")

list(grouped.keys())[:8]


Reading CSV: /Users/ujwalneethipudi/Downloads/European Political Tech Landscape/map_data_demo.csv
Wrote JSON: /Users/ujwalneethipudi/Downloads/European Political Tech Landscape/src/data/companies.json
Wrote JSON: /Users/ujwalneethipudi/Downloads/European Political Tech Landscape/public/companies.json


['Data Analytics & Modeling',
 'Engagement & Mobilisation',
 'Fundraising',
 'GovTech / Civic Infrastructure',
 'Messaging & Media',
 'Organisational Infrastructure',
 'Research & Insights',
 'Voting Tech']